In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from policy import Policy
from langchain_community.embeddings import OllamaEmbeddings

policy = Policy(data_dir="storage/policy", embedding=OllamaEmbeddings(model='llama3'))
policy_tools = policy.get_tools()
policy_tools

{'lookup_policy_tool': LookupPolicyTool(policy=<policy.Policy object at 0x000002D027206310>)}

In [3]:
print(policy_tools['lookup_policy_tool'].invoke(input={'query': "حداکثر برای چند نفر میشه بلیط تهیه کرد؟"}))

FAQ_SUBJECT: قطار
FAQ_QUESTION:
در یک خرید، برای چند مسافر به‌صورت همزمان می‌توانم خرید کنم؟
FAQ_ANSWER:
<p>حداکثر بلیت قابل ارائه به هر کاربر غیر حضوری در هر خرید برای قطار های اتوبوسی و 4 تخته تعداد 4 بلیت ، قطار های 6 تخته تعداد 6 بلیت و واگن حمل خودرو تعداد 1 بلیت میباشد .</p><p><br>&nbsp;</p>

FAQ_SUBJECT: پرواز خارجی
FAQ_QUESTION:
خرید بلیط به‌صورت  گروهی چه شرایطی دارد؟
FAQ_ANSWER:
<p style="text-align:right;">برای مسافران بیشتر از 20 نفر می‌توانید با مرکز پشتیبانی تماس بگیرید و اطلاعات پرواز و مسافران خود را اعلام کنید تا درخواست خرید گروهی شما بررسی و بلیط‌ها تهیه شود.</p><ul><li style="text-align:justify;"><a href="tel:+982143900000">۰۲۱-۴۳۹۰۰۰۰۰ : شماره تماس</a></li></ul>

FAQ_SUBJECT: هتل داخلی
FAQ_QUESTION:
چه ساعتی می‌توانم اتاق را تحویل بگیرم و چه ساعتی باید اتاق را تحویل بدهم؟
FAQ_ANSWER:
<p style="text-align:right;">معمولا در اکثر هتل ها ساعت ورود 14 و خروج در ساعت 12 انجام میگردد .&nbsp;</p>

FAQ_SUBJECT: پرواز خارجی
FAQ_QUESTION:
چگونه می‌توانم در مورد مقررات پرواز ی

In [4]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model='llama3', num_ctx=8192, num_thread=8, temperature=0.0)

In [5]:
import os
os.environ["TAVILY_API_KEY"] = "<>"

from online_search import PersianTavilySearchTool

tools = [PersianTavilySearchTool(max_results=3, llm=llm)] + list(policy_tools.values())

def get_tool_description(tool):
    tool_params = [
        f"{name}: {info['type']} ({info['description']})"
        for name, info in tool.args.items()
    ]
    tool_params_string = ', '.join(tool_params)
    return (
        f"tool_name -> {tool.name}\n"
        f"tool_params -> {tool_params_string}\n"
        f"tool_description ->\n{tool.description}"
    )

print(get_tool_description(tools[0]))

tool_name -> tavily_search_tool
tool_params -> query: string (search query to look up)
tool_description ->
A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.


In [6]:
import json
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

class ToolMessage(HumanMessage):
    """Ollama does not support `tool` role and `ToolMessage`"""


tools = [PersianTavilySearchTool(max_results=3, llm=llm)] + list(policy_tools.values())

tool_descs = '\n\n'.join([get_tool_description(tool) for tool in tools])

prompt_template = \
f"""
You are a helpful Persian customer support assistant for Iran Airlines.
Use the provided tools to search for flights, company policies, and other information to assist the user's queries. 
When searching, be persistent. Expand your query bounds if the first search returns no results. 
If a search comes up empty, expand your search before giving up.

You have access to the following tools to get more information if needed:

{tool_descs}

You also have access to the history of privious messages.

Generate the response in the following json format.
{{
    "THOUGHT": "<you should always think about what to do>",
    "ACTION": "<the action to take, must be one tool_name from above tools>",
    "ACTION_PARAMS": "<the input parameters to the ACTION, it must be in json format complying with the tool_params>"
    "FINAL_ANSWER": "<a text containing the final answer to the original input question>",
    "FINAL_ANSWER_PERSIAN": "<the Persian translated version of the FINAL_ANSWER>"
}}
If you don't know the answer, you can take an action using one of the provided tools.
But if you do, don't take and action and leave the action-related attributes empty.
The values `ACTION` and `FINAL_ANSWER` cannot be filled at the same time.

Always make sure that your output is a json complying with above format.
Do NOT add anything before or after the json response.
"""

user_question = "حداکثر برای چند نفر میتوان بلیط تهیه کرد؟"
messages = [
    SystemMessage(prompt_template),
    HumanMessage(user_question),
    AIMessage(json.dumps({
        "THOUGHT": "Let me check our policies on booking multiple tickets.", 
        "ACTION": "lookup_policy_tool", 
        "ACTION_PARAMS": {"query": "maximum number of passengers for a single booking"} 
    })),
    ToolMessage(
        "Here is the tool results:\n" +
        tools[1].invoke(input={"query": "maximum number of passengers for a single booking"})
    ),
]

# print(messages[-1].content)
print(llm.invoke(messages).content)

{"THOUGHT": "I think I can find the answer to your question about booking multiple tickets.", 
"ACTION": "", 
"ACTION_PARAMS": {}, 
"FINAL_ANSWER": "According to our policies, you can book a maximum of 9 tickets for international flights. However, please note that this may vary depending on the specific flight and travel dates. I recommend checking with our customer support team or consulting our website for more information.", 
"FINAL_ANSWER_ PERSIAN": ""}


In [7]:
user_question = "هزینه اضافه بار برای مسافرت خارجی چقدر است؟"
messages = [
    SystemMessage(prompt_template),
    HumanMessage(user_question),
    AIMessage(json.dumps({
        "THOUGHT": "Let's check the child fare policy for Iran Airlines.", 
        "ACTION": "lookup_policy_tool", 
        "ACTION_PARAMS": {"query": "excess baggage fee for international flights"} 
    })),
    ToolMessage(
        "Here is the tool results:\n" +
        tools[1].invoke(input={'query': "excess baggage fee for international flights"})
    ),
    AIMessage(json.dumps({
        "THOUGHT": "The provided tools didn't have the information about excess baggage fee for international flights.", 
        "ACTION": "tavily_search_tool", 
        "ACTION_PARAMS": {"query": "excess baggage fee for international flights Iran Airlines"} 
    })),
    ToolMessage(
        "Here is the tool results:\n" +
        tools[0].invoke(input={'query': "excess baggage fee for international flights Iran Airlines"})
    ),
    # AIMessage(json.dumps({
    #     "THOUGHT": "The search result suggests that Iran Airlines has a policy for excess baggage fees, but it's not explicitly stated.", 
    #     "ACTION": "lookup_policy_tool", 
    #     "ACTION_PARAMS": {"query": "excess baggage fee for international flights Iran Airlines"} 
    # })),
    # ToolMessage(
    #     "Here is the tool results:\n" +
    #     tools[1].invoke(input={'query': "excess baggage fee for international flights Iran Airlines"})
    # ),
]

# print(messages[-1].content)
print(llm.invoke(messages).content)

{"THOUGHT": "The search results indicate that the excess baggage fee for international flights from Iran to Turkey and vice versa depends on the airline and ticket type.", 
"ACTION": "", 
"ACTION_PARAMS": {}, 
"FINAL_ANSWER": "According to the report, purchasing excess baggage for adult and child passengers on international flights 'Hema' is possible in package form. The additional fee per kilogram of excess baggage from Turkish airports to Iranian airports is 100,000 tomans plus a 9% tax increase, which is approximately 9 euros.", 
"FINAL_ANSWER_PERSIAN": "بر اساس گزارش اداره روابط عمومی، خرید اضافه بار برای مسافران بزرگسال و کودک در پروازهای بین‌المللی 'هما' در بسته‌بندی امکان‌پذیر است. هزینه اضافی هر کیلوگرم اضافه بار از فرودگاه‌های ترکیه به فرودگاه‌های ایران ۱۰۰ هزار تومان با افزایش مالی ۹ درصد است که تقریباً ۹ یورو است."}


In [8]:
user_question = "نرخ بلیط هواپیما برای کودکان زیر ۱۰ سال چقدر است؟"
messages = [
    SystemMessage(prompt_template),
    HumanMessage(user_question),
    AIMessage(json.dumps({
        "THOUGHT": "Let's check the child fare policy for Iran Airlines.", 
        "ACTION": "lookup_policy_tool", 
        "ACTION_PARAMS": {"query": "child fare policy iran airlines"} 
    })),
    ToolMessage(
        "Here is the tool results:\n" +
        tools[1].invoke(input={'query': "child fare policy iran airlines"})
    ),
]

# print(messages[-1].content)
print(llm.invoke(messages).content)

{"THOUGHT": "The child fare policy for Iran Airlines is mentioned in the FAQ answer.", 
"FINAL_ANSWER": "According to the FAQ, the child fare for children under 10 years old is half of the adult ticket price.", 
"FINAL_ANSWER_PERSIAN": "بر اساس پاسخFAQ، نرخ بلیط کودک برای کودکان زیر ۱۰ سال نصف قیمت بلیط بزرگسال است."}


In [9]:
user_question = "نرخ بلیط هواپیما برای کودکان زیر ۱۰ سال چقدر است؟"
messages = [
    SystemMessage(prompt_template),
    HumanMessage(user_question),
    AIMessage(json.dumps({
        "THOUGHT": "Let's check the child fare policy for Iran Airlines.", 
        "ACTION": "lookup_policy_tool", 
        "ACTION_PARAMS": {"query": "child fare policy for children under 10 years old"} 
    })),
    ToolMessage(
        "Here is the tool results:\n" +
        tools[1].invoke(input={'query': "child fare policy for children under 10 years old"})
    ),
]

# print(messages[-1].content)
print(llm.invoke(messages).content)

{"THOUGHT": "Unfortunately, the provided tools didn't have any information about child fare policy for children under 10 years old.", 
"FINAL_ANSWER": "We don't have a specific policy for child fares for children under 10 years old. However, we can offer you a special rate for children in this age group. Please contact our customer support team to get more information and make the necessary arrangements.", 
"FINAL_ANSWER_PERSIAN": "ما سیاست خاصی برای نرخ بلیط کودکان زیر ۱۰ سال نداریم. اما می‌توانیم نرخ ویژه‌ای برای کودکان در این گروه سنی ارائه دهیم. لطفا با تیم پشتیبانی客رو تماس بگیرید تا اطلاعات بیشتر و تنظیمات لازم را انجام دهید."}


In [10]:
from database import Database

database = Database(data_dir="storage/database")

In [11]:
from flight import FlightManager

flight_manager = FlightManager(database)

passenger_id = '3442 587242'
print(flight_manager.fetch_user_flight_information(passenger_id))
print(len(flight_manager.search_flights()))

[{'ticket_no': '7240005432906569', 'book_ref': 'C46E9F', 'flight_id': 19250, 'flight_no': 'LX0112', 'departure_airport': 'CDG', 'arrival_airport': 'BSL', 'scheduled_departure': '2024-05-10 00:20:04.578832-04:00', 'scheduled_arrival': '2024-05-10 01:50:04.578832-04:00', 'seat_no': '18E', 'fare_conditions': 'Economy'}]
20


In [12]:
import uuid

from flight import FlightManager

flight_manager = FlightManager(database)
flight_tools = flight_manager.get_tools()

config = {
    'configurable': {
        'passenger_id': '3442 587242',
        'thread_id': str(uuid.uuid4()),
    }
}

flight_tools['fetch_user_flight_information_tool'].invoke(input={}, config=config)

'[{"ticket_no": "7240005432906569", "book_ref": "C46E9F", "flight_id": 19250, "flight_no": "LX0112", "departure_airport": "CDG", "arrival_airport": "BSL", "scheduled_departure": "2024-05-10 00:20:04.578832-04:00", "scheduled_arrival": "2024-05-10 01:50:04.578832-04:00", "seat_no": "18E", "fare_conditions": "Economy"}]'

In [13]:
from llm_translation import translate_to_persian

tools = [
    PersianTavilySearchTool(max_results=3, llm=llm)
] + list(policy_tools.values()) + list(flight_tools.values())

tool_descs = '\n\n'.join([get_tool_description(tool) for tool in tools])

prompt_template = \
f"""
You are a helpful Persian customer support assistant for Iran Airlines.
Use the provided tools to search for flights, company policies, and other information to assist the user's queries. 
When searching, be persistent. Expand your query bounds if the first search returns no results. 
If a search comes up empty, expand your search before giving up.

You have access to the following tools to get more information if needed:

{tool_descs}

You also have access to the history of privious messages.

Generate the response in the following json format.
{{
    "THOUGHT": "<you should always think about what to do>",
    "ACTION": "<the action to take, must be one tool_name from above tools>",
    "ACTION_PARAMS": "<the input parameters to the ACTION, it must be in json format complying with the tool_params>"
    "FINAL_ANSWER": "<a text containing the final answer to the original input question>",
}}
If you don't know the answer, you can take an action using one of the provided tools.
But if you do, don't take and action and leave the action-related attributes empty.
The values `ACTION` and `FINAL_ANSWER` can never ever be filled at the same time.

Always make sure that your output is a json complying with above format.
Do NOT add anything before or after the json response.
"""

config = {
    'configurable': {
        'passenger_id': '3442 587242',
        'thread_id': str(uuid.uuid4()),
    }
}

user_question = "امکانش هست اطلاعات پروازم رو بگید؟"
user_question = "من فراموش کردم که اطلاعات پروازم چی بوده"

messages = [
    SystemMessage(prompt_template),
    HumanMessage(user_question),
    AIMessage(json.dumps({
        "THOUGHT": "You're asking about flight information, let me see what I can do for you.", 
        "ACTION": "fetch_user_flight_information_tool", 
        "ACTION_PARAMS": {} 
    })),
    ToolMessage(
        "Here is the tool results:\n" +
        tools[2].invoke(input={}, config=config)
    ),
]

# print(messages[-1].content)
raw_result = llm.invoke(messages, config=config).content
print(raw_result)
result = json.loads(raw_result)
if result.get('FINAL_ANSWER'):
    print(translate_to_persian(result['FINAL_ANSWER'], llm))

{"THOUGHT": "I've got the flight information for you! Let me summarize it.", 
"FINAL_ANSWER": "Your ticket number is 7240005432906569, and your flight details are: Flight LX0112 from CDG to BSL on May 10th, 2024. Your seat number is 18E, and the fare conditions are Economy."}
شماره بلیت شما ۷۲۴۰۰۰۵۴۳۲۹۰۶۵۶۹ است و جزئیات پروازتان عبارتند از: پرواز LX0112 از CDG به BSL در تاریخ ۱۰ ماه مه ۲۰۲۴. شماره صندلی شما ۱۸E است و شرایط قیمتی شما Economy است.


In [14]:
user_question = "اطلاعات پروازهای به مقصد BSL رو با جزئیات کامل بهم بده"

messages = [
    SystemMessage(prompt_template),
    HumanMessage(user_question),
    AIMessage(json.dumps({
        "THOUGHT": "The user wants to search for flights to Basel (BSL).", 
        "ACTION": "search_flights_tool", 
        "ACTION_PARAMS": {'departure_airport': '', 'arrival_airport': 'BSL', 'start_time': '', 'end_time': '', 'limit': 10}
    })),
    ToolMessage(
        "Here is the tool results:\n" +
        tools[3].invoke(input={'departure_airport': '', 'arrival_airport': 'BSL', 'start_time': '', 'end_time': '', 'limit': 10}, config=config)
    ),
]

# print(messages[-1].content)
raw_result = llm.invoke(messages, config=config).content
print(raw_result)
result = json.loads(raw_result)
if result.get('FINAL_ANSWER'):
    print(translate_to_persian(result['FINAL_ANSWER'], llm))

{"THOUGHT": "The user wants to search for flights to Basel (BSL). The tool results show a list of available flights.", 
"ACTION": "", 
"ACTION_PARAMS": {}, 
"FINAL_ANSWER": "Here are the flight details: \n\n1. Flight AA0055 from CEB to BSL on 2024-05-30 at 15:35.\n2. Flight TG0023 from DME to BSL on 2024-06-06 at 13:40.\n3. Flight AF0137 from HAM to BSL on 2024-05-01 at 18:15 (arrived).\n4. Flight LH0130 from HAM to BSL on 2024-05-01 at 01:20 (arrived), and again on 2024-05-16 at 01:20.\n5. Flight TK0081 from HAM to BSL on 2024-06-01 at 21:15, and again on 2024-05-03 at 21:15 (arrived).\n6. Flight QR0055 from HAM to BSL on 2024-06-01 at 18:30.\nPlease let me know if you would like more information about any of these flights or if you would like to book one."}
مشاهده جزئیات پروازها:

1. پرواز AA0055 از CEB به BSL در تاریخ 2024-05-30 ساعت 15:35.
2. پرواز TG0023 از DME به BSL در تاریخ 2024-06-06 ساعت 13:40.
3. پرواز AF0137 از HAM به BSL در تاریخ 2024-05-01 ساعت 18:15 (پرواز رسید).
4. پروا

In [15]:
user_question = "اطلاعات همه پروازهای فردا رو میخواستم"


messages = [
    SystemMessage(prompt_template),
    HumanMessage(user_question),
    AIMessage(json.dumps({
        "THOUGHT": "The user wants to search for flights to destination BSL.", 
        "ACTION": "search_flights_tool", 
        "ACTION_PARAMS": {"departure_airport": "", "arrival_airport": "", "start_time": "2023-03-17T00:00:00+00:00", "end_time": "2023-03-18T23:59:59+00:00", "limit": 100}
    })),
    ToolMessage(
        "Here is the tool results:\n" +
        tools[3].invoke(input={"departure_airport": "", "arrival_airport": "", "start_time": "2023-03-17T00:00:00+00:00", "end_time": "2023-03-18T23:59:59+00:00", "limit": 100}, config=config)
    ),
]

# print(messages[-1].content)
raw_result = llm.invoke(messages, config=config).content
print(raw_result)
result = json.loads(raw_result)
if result.get('FINAL_ANSWER'):
    print(translate_to_persian(result['FINAL_ANSWER'], llm))

{"THOUGHT": "No flights found for tomorrow. Let's try to expand the search range.",  "ACTION": "search_flights_tool",  "ACTION_PARAMS": {"departure_airport": "", "arrival_airport": "BSL", "start_time": "2023-03-16T00:00:00+00:00", "end_time": "2023-03-19T23:59:59+00:00", "limit": 100}}


In [18]:
user_question = "امکانش هست تیکتم رو به شماره ۷۲۴۰۰۰۵۴۳۲۹۰۶۵۶۹ کنسل کنم؟"


messages = [
    SystemMessage(prompt_template),
    HumanMessage(user_question),
    AIMessage(json.dumps({
        "THOUGHT": "Let's check if the ticket can be cancelled and what are the possible actions.", 
        "ACTION": "cancel_ticket_tool", 
        "ACTION_PARAMS": {"ticket_no": "7240005432906569"},
    })),
    ToolMessage(
        "Here is the tool results:\n" +
        tools[5].invoke(input={"ticket_no": "7240005432906569"}, config=config)
    ),
]

# print(messages[-1].content)
raw_result = llm.invoke(messages, config=config).content
print(raw_result)
result = json.loads(raw_result)
if result.get('FINAL_ANSWER'):
    print(translate_to_persian(result['FINAL_ANSWER'], llm))

{"THOUGHT": "The ticket has been successfully cancelled.", "FINAL_ANSWER": "Ticket successfully cancelled."}
بلیط با موفقیت لغو شد.
